## Gravitational Orbits

# Cameron Kovanda

# 5300 Final Problem 2

# Dr. Furnstahl

This notebook creates a GravitationalOrbits class that solves the differential equations for gravitational attraction between 2 bodies (e.g. Earth and Sun) in Cartesian coordinates. It also provides a look into how it can be reduced into the orbits discussed in class when one of the bodies is very heavy and you are in its rest frame.

In [ ]:
%matplotlib inline

import numpy as np
from scipy.integrate import solve_ivp

import matplotlib.pyplot as plt
from IPython.display import Image

In [ ]:
Image(url='https://www.solarsystemscope.com/spacepedia/images/articles_2017/orbital-and-rotational-characteristics-of-earth_02.jpg')

In [ ]:
class GravitationalOrbits():
    """
    This class implements the parameters and Lagrange's equations for two particles orbiting
    with gravitational attraction
    
    Parameters
    ----------
    m1 : float
         mass of particle 1
    m2 : float
         mass of particle 2
    G : float
        Gravitational constant
    
    Methods
    -------
    dy_dt(t,y)
        Returns the right side of the differential equation in vector y, 
        given time t and the corresponding value of y.
    """
    def __init__(self, m1=1., m2=1., G=1.):
        self.m1 = m1
        self.m2 = m2
        self.G = G
        
    def dz_dt(self, t, z):
        """
        This function returns the right hand side of the differential equation:
        [dz/dt and d^2z/dt^2]
        
        Parameters
        ----------
        t : float
            time
        z : float
            8-component vector with:
            
            z[0] = x1(t) and z[1] = x1d(t)
            z[2] = y1(t) and z[3] = y1d(t)
            z[4] = x2(t) and z[5] = x2d(t)
            z[6] = y2(t) and z[7] = y2d(t)
        
        Returns
        -------
        """
        r12 = np.sqrt( (z[0]-z[4])**2 + (z[2]-z[6])**2 )
        return [ \
                z[1], self.G * self.m2 * (z[4]-z[0]) / r12**3, \
                z[3], self.G * self.m2 * (z[6]-z[2]) / r12**3, \
                z[5], -self.G * self.m1 * (z[4]-z[0]) / r12**3, \
                z[7], -self.G * self.m1 * (z[6]-z[2]) / r12**3, \
               ]
    
    def solve_ode(self, t_pts, z_0,
                 abserr=1.0e-10, relerr=1.0e-10):
        """
        Solve the ODE given the initial conditions of the system.
        Specify small absolute and relative error to get higher precision
        """
        solution = solve_ivp(self.dz_dt, (t_pts[0], t_pts[-1]),
                             z_0, t_eval=t_pts, method='RK23',
                             atol=abserr, rtol=relerr)
        
        x1, x1d, y1, y1d, x2, x2d, y2, y2d = solution.y
        
        return x1, x1d, y1, y1d, x2, x2d, y2, y2d

In [ ]:
def plot_y_vs_x(x, y, axis_labels=None, label=None, title=None, 
                color=None, linestyle=None, semilogy=False, loglog=False,
                ax=None):
    """
    Generic plotting function: return a figure axis with a plot of y vs. x,
    with line color and style, title, axis labels, and line label
    """
    if ax is None:        # if the axis object doesn't exist, make one
        ax = plt.gca()

    if (semilogy):
        line, = ax.semilogy(x, y, label=label, 
                            color=color, linestyle=linestyle)
    elif (loglog):
        line, = ax.loglog(x, y, label=label, 
                          color=color, linestyle=linestyle)
    else:
        line, = ax.plot(x, y, label=label, 
                    color=color, linestyle=linestyle)

    if label is not None:    # if a label if passed, show the legend
        ax.legend()
    if title is not None:    # set a title if one if passed
        ax.set_title(title)
    if axis_labels is not None:  # set x-axis and y-axis labels if passed  
        ax.set_xlabel(axis_labels[0])
        ax.set_ylabel(axis_labels[1])

    return ax, line

In [ ]:
def start_stop_indices(t_pts, plot_start, plot_stop):
    start_index = (np.fabs(t_pts-plot_start)).argmin()  # index in t_pts array 
    stop_index = (np.fabs(t_pts-plot_stop)).argmin()  # index in t_pts array 
    return start_index, stop_index

## Make Simple Orbit Plots

In [ ]:
# Labels for individual plot axes
orbit_vs_time_labels = (r'$x$', r'$y$')

# Common plotting time (generate the full time then use slices)
t_start = 0.
t_end = 10.
delta_t = 0.01

t_pts = np.arange(t_start, t_end+delta_t, delta_t)  

In [ ]:
# Initial parameters
t_start = 0.
t_end = 10.
delta_t = 0.01

m1 = 1.
m2 = 10.
G = 1.

# Instantiate a set of orbits

o1 = GravitationalOrbits(m1,m2,G)

# Initial conditions with one particles velocity equal to zero

x1_0, x1d_0 = 1., -1.
y1_0, y1d_0 = 1., 1.
x2_0, x2d_0 = -(m1/m2) * x1_0, -(m1/m2) * x1d_0
y2_0, y2d_0 = -(m1/m2) * y1_0, -(m1/m2) * y1d_0

z_0 = [x1_0, x1d_0, y1_0, y1d_0, x2_0, x2d_0, y2_0, y2d_0]

x1, x1d, y1, y1d, x2, x2d, y2, y2d = o1.solve_ode(t_pts, z_0)

# Start the plot!

fig = plt.figure(figsize=(5,5))
overall_title = 'Simple Gravitational Orbit  '
fig.suptitle(overall_title, va='baseline')

# First Orbit Plot 
    
ax = fig.add_subplot(1,1,1)                  

start, stop = start_stop_indices(t_pts, t_start, t_end)
ax.plot(x1, y1, color='green', label=r'$m1$')
ax.plot(x2, y2, color='blue', label=r'$m2$')    
ax.legend()
ax.set_aspect(1)

fig.tight_layout()
fig.savefig('Gravitational_Orbits_1.png', bbox_inches='tight')  

Now we can test with a few other initial conditions

In [ ]:
# Initial parameters
t_start = 0.
t_end = 10.
delta_t = 0.01

m1 = 1.
m2 = 20.
G = 1.

# Instantiate a set of orbits

o1 = GravitationalOrbits(m1,m2,G)

# Initial conditions with one particles velocity equal to zero

x1_0, x1d_0 = 1., 1.
y1_0, y1d_0 = 1., -1.
x2_0, x2d_0 = -(m1/m2) * x1_0, -(m1/m2) * x1d_0
y2_0, y2d_0 = -(m1/m2) * y1_0, -(m1/m2) * y1d_0

z_0 = [x1_0, x1d_0, y1_0, y1d_0, x2_0, x2d_0, y2_0, y2d_0]

x1, x1d, y1, y1d, x2, x2d, y2, y2d = o1.solve_ode(t_pts, z_0)

# Start the plot!

fig = plt.figure(figsize=(5,5))
overall_title = 'Simple Gravitational Orbit  '
fig.suptitle(overall_title, va='baseline')

# Second Orbit Plot 
    
ax = fig.add_subplot(1,1,1)                  

start, stop = start_stop_indices(t_pts, t_start, t_end)
ax.plot(x1, y1, color='green', label=r'$m1$')
ax.plot(x2, y2, color='blue', label=r'$m2$')    
ax.legend()
ax.set_aspect(1)

fig.tight_layout()
fig.savefig('Gravitational_Orbits_2.png', bbox_inches='tight')  